In [12]:
import pandas as pd
from scipy.spatial import distance
from sqlalchemy import create_engine

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [13]:
import os

import pymysql
import pandas as pd

from sqlalchemy import create_engine
from dotenv import load_dotenv


In [14]:
# setup env
load_dotenv()

True

In [15]:
df_feature_vector = pd.read_parquet('data/feature_vector_num.parquet')
df_feature_vector_with_text = pd.read_parquet('data/feature_vector.parquet')
df_tag = pd.read_parquet('data/tag.parquet')
df_feature_vector_with_text.shape

(134, 80)

In [16]:
# setup db
user = os.getenv('RDS_USERNAME')
password = os.getenv('RDS_PASSWORD')
host = os.getenv('RDS_HOSTNAME')
db = os.getenv('RDS_DB_NAME')
port = os.getenv('RDS_PORT')

db_string = "postgresql://{}:{}@{}:{}/{}".format(user,password,host,port,db)
sqlEngine = create_engine(db_string)

In [17]:
df_tag = pd.read_parquet('data/tag.parquet')
df_tag = df_tag[['name', 'category_id']]
df_tag['weight'] = df_tag['category_id'].apply(lambda x: 3 if x == 9 else 0.3 if x == 1 else 1)

In [18]:
df_all_features = pd.DataFrame(df_feature_vector.columns.to_list(), columns=['name'])

In [19]:
df_merge_tag_features = pd.merge(df_tag, df_all_features, on='name')

allFeatures = df_feature_vector.columns.to_list()
df_name = df_merge_tag_features['name']

for i in df_name.to_numpy():
    allFeatures.remove(i)

for i in allFeatures:
    df_merge_tag_features = df_merge_tag_features.append({'name': i, 'weight': 0.3}, ignore_index=True)

array = df_feature_vector.columns.to_list()
array.remove('id')
array.remove('Mit Abstand')
array.remove('Videokonferenz')
array.remove('Pfadfinder')
array.remove('Rover')

weight = df_merge_tag_features['weight'].to_numpy()
array_labels = df_merge_tag_features['name'].to_numpy()

(134, 61)

In [20]:
heimabend_index = 5

for heimabend_index in range(0, df_feature_vector.shape[0]):
    print('heimabend_index')
    print(heimabend_index)
    print('heimabend_index')
    array_labels = df_feature_vector.columns.to_list()

    df_feature_vector_with_text['index'] = df_feature_vector_with_text.index
    title_array = df_feature_vector_with_text[['id', 'title']]
    title_list = title_array.values.tolist()

    feature_list = df_feature_vector[array_labels].values.tolist()

    ary_1 = feature_list[heimabend_index]

    feature_ary = []
    for index, element in enumerate(feature_list):
        score_cosine = distance.cosine(ary_1, element, weight)
        feature_ary.append([score_cosine])
        # print(score_cosine)

    df_1 = pd.DataFrame(feature_ary, columns=['score'])
    df_2 = pd.DataFrame(title_list, columns=['event_score_id', 'title'])
    df = pd.concat([df_1, df_2], axis=1,).sort_values(by=['score']).iloc[1:15]
    df['event_id'] = df_feature_vector_with_text.iloc[heimabend_index]['id']
    df['is_public'] = True
    df[['event_id', 'event_score_id', 'score', 'is_public']].to_sql('basic_nextbestheimabend', con=sqlEngine, if_exists='append', index=False)
    # print(df[['score', 'title', 'event_score_id', 'event_id']])

heimabend_index
0
heimabend_index
heimabend_index
1
heimabend_index
heimabend_index
2
heimabend_index
heimabend_index
3
heimabend_index
heimabend_index
4
heimabend_index
heimabend_index
5
heimabend_index
heimabend_index
6
heimabend_index
heimabend_index
7
heimabend_index
heimabend_index
8
heimabend_index
heimabend_index
9
heimabend_index
heimabend_index
10
heimabend_index
heimabend_index
11
heimabend_index
heimabend_index
12
heimabend_index
heimabend_index
13
heimabend_index
heimabend_index
14
heimabend_index
heimabend_index
15
heimabend_index
heimabend_index
16
heimabend_index
heimabend_index
17
heimabend_index
heimabend_index
18
heimabend_index
heimabend_index
19
heimabend_index
heimabend_index
20
heimabend_index
heimabend_index
21
heimabend_index
heimabend_index
22
heimabend_index
heimabend_index
23
heimabend_index
heimabend_index
24
heimabend_index
heimabend_index
25
heimabend_index
heimabend_index
26
heimabend_index
heimabend_index
27
heimabend_index
heimabend_index
28
heimabend_i

In [21]:
    # df[['event_id', 'event_score_id', 'score',]].to_sql('basic_nextbestheimabend', con=sqlEngine, if_exists='append', index=False)
    # print(df[['score', 'title', 'event_score_id', 'event_id']])